# Using stacopy

`stacopy` guides the use of the $k$-means unsupervised clustering method ("`k-means`"; MacQueen 1967; Lloyd 1982) for samples in which the true clustering structure is not known. `k-means`, which partitions samples into $k$ compact, spherical clusters, uses randomised initialisations to overcome its local dependency. Different initialisations lead to different clustering outcomes, and clustering outcomes may be more or less varied at different values of $k$. Therefore, one needs to be able to not only a) identify the best clustering outcome at a given value of $k$, but b) identify the best value of $k$ for modelling the clustering structure of a sample. `stacopy` does both, identifying the most compact clustering outcomes and the most stables values of $k$.

Necessary imports and setup:

In [1]:
import numpy as N
import matplotlib as M
M.use('Agg')
import matplotlib.pyplot as MP
import stacopy as S

if __name__ == '__main__': # required to run parallel code in parallel; any code using stacopy functions should begin with this!

    MP.rcParams.update({'figure.figsize':[4,2], 'figure.dpi': 300, 'font.family': 'Times New Roman', 'font.size': 10, 'text.usetex': True, 'figure.autolayout': True})

We'll cluster a simple, simulated sample to begin with. This will demonstrate the use of stability for selecting good values of $k$. The simulation consists of 5000 points distributed equally over five 2D Gaussian functions ($\sigma = 0.3$), centred at the vertices of a unit regular pentagon. It consists of five true clusters, and is shown in the figure below:

<img src='pics/sim_black.png' style="width:40%">

We'll load the simulation, and run `k-means` 25 times each at four different values of $k$.

In [2]:
    # input data must be clustering features only; no IDs, flags, etc.
    # hence, we chop off the IDs in the third column of sim.txt for clustering
    data = N.genfromtxt('sim.txt', delimiter = ',')[:,:2]

    k = range(3,8)
    init = 25

    staco = S.STACO(data, k = k, init = init)

Clustering...
k = 3
k = 4
k = 5
k = 6
k = 7
Done.
Measuring stabilities...
k = 3
k = 4
k = 5
k = 6
k = 7
Done.


The function `STACO()` returns a (`init*len(k),3`)-shape array of stabilities (median $V$), compactnesses ($\phi$), and values of $k$ of every clustering outcome we generate. For now, we're mostly interested in the ability of `STACO()` to use stability to point us to the right value of $k$ for describing the simulation, so we plot a stability map:

In [3]:
    fig, ax = MP.subplots() # plotting the stability map
    
    for i in k:
        cvs = staco[staco[:,2] == i,0].reshape(-1,1)

        hist, bins = N.histogram(cvs, bins = N.arange(0.0,1.01,0.01))
        hist = (hist.astype(float) / float(max(hist))) * 0.8
    
        ax.plot([0.0, 1.01], [i, i], c = 'k', ls = '-', lw = 1.5, zorder = 3) # baselines
        ax.bar(bins[1:] - 0.005, hist, width = 0.01, bottom = i, color = 'k')
        ax.plot([N.mean(cvs),N.mean(cvs)],[i, i + 1], 'r-')
    
    ax.set(yticks = k, xlim = (0.5, 1.01), ylim = (min(k) - 1, max(k) + 1),
           xlabel = 'Median $V$', ylabel = '$k$')
    fig.savefig('pics/staco_sim.png')

The stability map for the simulation looks like this.

<img src='pics/staco_sim.png' style="width:60%">

The key element in interpreting this plot is the gap in the distributions of the stabilities of clustering outcomes. The distribution of stabilities at $k = 5$ shows that `k-means` has converged to the same stable clustering outcome following all 25 initialisations. Distributions of stabilities at other values of $k$, meanwhile, show that there is not objectively correct way to divide the five true clusters into an alternative number of `k-means` clusters. Hence, we get a clear indication of the true clustering structure of the simulation. 

We'll now have a go at a more complicated sample. We'll cluster the five-dimensional galaxy sample used in Turner et al. (2018). We can't visualise this sample, so we have to trust the stability map to tell us what value(s) of $k$ are good. We'll run `k-means` 100 times each at 14 different values of $k$. We'll also make sure to save the labels so we can retrieve them again later.

In [5]:
    # first column in gal.txt is GAMA survey IDs
    # hence we chop them off for clustering
    # note that the data in gal.txt is Z-scored
    data = N.genfromtxt('gal.txt', delimiter = ',')[:,1:] 
    
    k = range(2,16) # values of k
    init = 100 # number of k-means initialisations at each k

    staco = S.STACO(data, k, init, save_lbls = True) # saving labels for later

    N.savetxt('staco_gal.txt', staco, delimiter = ',') # saving staco info for later

Clustering...
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
Done.
Measuring stabilities...
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
Done.


We can now plot the stability map for this galaxy sample:

In [6]:
    MP.rcParams.update({'figure.figsize':[4,4]})
    
    fig, ax = MP.subplots() # plotting the stability map
    
    for i in k:
        cvs = staco[staco[:,2] == i,0].reshape(-1,1)

        hist, bins = N.histogram(cvs, bins = N.arange(0.0,1.01,0.01))
        hist = (hist.astype(float) / float(max(hist))) * 0.8
    
        ax.plot([0.0, 1.01], [i, i], c = 'k', ls = '-', lw = 1.5, zorder = 3) # baselines
        ax.bar(bins[1:] - 0.005, hist, width = 0.01, bottom = i, color = 'k')
        ax.plot([N.mean(cvs),N.mean(cvs)],[i, i + 1], 'r-')
    
    ax.set(yticks = k, xlim = (0.5, 1.01), ylim = (min(k) - 1, max(k) + 1),
           xlabel = 'Median $V$', ylabel = '$k$')
    fig.savefig('pics/staco_gal.png')

Which looks like this.

<img src='pics/staco_gal.png' style="width:60%">

Here we find stable clustering at $k = 2,3,5,$ and $6$. There are multiple stable values of $k$ because the sample is asymmetrical in the five-dimensional feature space in which we cluster it, and because it has a hierarchical clustering structure. See Turner et al. (2018) for more info. 

Finally, we'll look at grabbing cluster labels, which can be used to analyse clusters. We'll go with $k = 6$, which we endorse in Turner et al. (2018) as being the best value of $k$ for describing the galaxy sample in terms of both capturing the broad aspects of the bimodality and making finer distinctions between different sub-types of galaxies at the same time. The best solution of the 100 we generated at $k = 6$ is the most compact (i.e. that with the lowest $\phi$). Hence:

In [7]:
lbls = N.genfromtxt('lbls_k6.txt', delimiter = ',') # loading k = 6 labels

ksix = staco[staco[:,2] == 6,:] # grabbing all k = 6 solutions

best = N.where(ksix[:,2] == min(ksix[:,2]))[0][0] # finding lowest compactnesses
clst = lbls[:,best] # selecting solution with lowest compactness

Note that the most compact solution might emerge several times, which is why we select the row containing the first instance (i.e. `...)[0][0]`) that satisfies this condition.

The final labels in `clst` are in the same order as the galaxies in the input table `gal.txt`, so it's straightforward to map the labels back onto the input galaxies. From there, you can interpret and visualise clusters by making plots like the one shown below, in which we show clusters from four different values of $k$ as contours in the colour-mass plane (two of the clustering features, using the non-Z-scored data). 

<img src='pics/colmass.png' style="width:100%">